#Introdução
Esse relatório documenta o processo de ponta a ponta para tratamento, análise e engenharia de features de um dataset de transações de clientes. A partir de dados brutos da Delta Table **default.transacoes_clientes**, foi executado um pipeline em _PySpark_ que envolveu a correção de tipos de dados, limpeza, e a criação de um conjunto robusto de features comportamentais. A análise exploratória subsequente revelou uma forte assimetria na distribuição dos valores de transação e indicou correlações importantes entre as categorias de gastos e o status de inadimplência. O case culminou na criação da tabela **default.clientes_features**, um asset de dados enriquecido e pronto para uma possível fase de desenvolvimento de um modelo de _machine learning_ para previsão de risco.

#1. As Transformações Realizadas (O Pipeline de Dados)
O pipeline foi estruturado para garantir a máxima qualidade e relevância dos dados para análise e modelagem.

##1.1. Ingestão e Preparação Inicial
Fonte: Os dados foram lidos da tabela Delta **default.transacoes_clientes**.

###Correção de Tipos (Type Casting):
A primeira análise do schema mostrou a necessidade de ajustes. A operação de cast foi aplicada para converter colunas para tipos mais apropriados, como Int para **id_cliente** e inadimplente, e Float para valor_transacao. Essa etapa é fundamental para otimizar o uso de memória e garantir a precisão em operações aritméticas e agregações. O schema foi transformado da seguinte forma:
Antes: **id_cliente**: long, **valor_transacao**: double, **inadimplente**: long
Depois: **id_cliente**: int, **valor_transacao**: float, **inadimplente**: int

##1.2. Qualidade e Transformação dos Dados
####Tratamento de Valores Ausentes: A estratégia de imputação foi aplicada para preservar o máximo de informações:
Para **valor_transacao**, valores nulos foram substituídos por **0.0**. A justificativa é que um valor nulo pode representar uma transação falha, estornada ou sem custo, onde o valor zero é a representação financeira mais adequada.
Para categoria, valores nulos foram preenchidos com a string "**desconhecida**". Isso evita a perda de registros e cria um grupo específico que pode ser analisado para identificar possíveis falhas na categorização ou padrões em transações não classificadas.
Remoção de Duplicatas: Utilizando **.dropDuplicates()**, foram eliminados todos os registros que eram idênticos em todas as colunas. Este passo é vital para a integridade das análises, garantindo que cada transação seja contada apenas uma vez em cálculos de frequência, soma e médias.

##1.3. Engenharia de Features
A criação de novas variáveis (_features_) foi realizada em duas fases para enriquecer o dataset.

####Features Agregadas Iniciais:
**num_transacoes**: Criada a partir de uma operação **groupBy("id_cliente").agg(count("*"))**, esta feature serve como um indicador primário do volume de atividade de cada cliente.
**media_mensal**: Calculada agrupando por cliente e **ano_mes** (coluna de apoio), essa feature ajuda a normalizar os gastos e identificar o comportamento financeiro médio mensalmente.

####Features Avançadas para Modelagem: Um conjunto final de features foi consolidado na tabela default.clientes_features.
Estas features foram projetadas para capturar diferentes dimensões do comportamento do cliente:
Recência, Frequência e Valor (_RFM-like_):
-**recencia**: Calculada como a diferença em dias entre a data da última transação no dataset e a última compra do cliente **(datediff(max_date, ultima_compra)**). É um forte indicador de quão ativo um cliente está.
-**total_gasto_ultimo_mes** e **qtd_transacoes_ultimo_mes**: Medem o comportamento de curto prazo, filtrando transações nos últimos 30 dias.
-**ticket_medio**: Média de **valor_transacao** por cliente, indicando o perfil de gasto por compra.

####Codificação Categórica (One-Hot Encoding):
A coluna categoria foi transformada utilizando uma operação de _pivot_. Isso resultou em um formato "_wide_", onde cada categoria de gasto se tornou uma coluna (ex: **cat_mercado**, **cat_lazer**) e o valor da célula representa a contagem de transações do cliente naquela categoria. Este formato é ideal para algoritmos de _machine learning_.

#2. Insights dos Dados (Análise Exploratória)
##2.1. Perfil Geral das Transações

O valor médio por transação é de R$ 29,32. No entanto, essa média é acompanhada por um alto desvio padrão de R$ 31,92, com valores que variam de R$ 0,00 até um máximo de R$ 395,78. Essa disparidade indica uma forte assimetria nos dados, onde a maioria das transações é de baixo valor, mas uma minoria de transações de alto valor eleva a média. A análise de frequência confirma este padrão de cauda longa, mostrando uma altíssima concentração de operações em valores inferiores a R$ 50, com a frequência diminuindo drasticamente à medida que o valor aumenta. Este comportamento é típico de dados de varejo, refletindo que a maior parte da atividade dos clientes é composta por compras rotineiras e de baixo custo.
Em relação ao engajamento, os clientes realizaram uma média de 6.07 transações no período analisado. A contagem variou de clientes com uma única transação até os mais ativos, com 13 transações, demonstrando diferentes níveis de fidelidade e atividade na plataforma.

####Análise de Inadimplência por Categoria
Para entender a relação entre o comportamento de consumo e o risco, foi realizada uma análise que quantifica as transações de clientes inadimplentes e não inadimplentes dentro de cada categoria. A principal descoberta é que o risco de inadimplência não é distribuído de forma homogênea. Observou-se que categorias associadas a despesas essenciais e recorrentes, como 'mercado' e 'transporte', tendem a ter uma proporção menor de transações de inadimplentes. Em contrapartida, categorias ligadas a gastos de maior valor ou de caráter não essencial, como 'educação', 'saúde' e 'lazer', demonstram uma proporção relativamente maior de inadimplência. Este insight é de alto valor estratégico, pois sugere que o perfil de consumo de um cliente pode ser um forte indicador preditivo de seu risco financeiro.

#3. Limitações e Sugestões de Melhorias
##3.1. Limitações Atuais
-Dados Demográficos: A principal limitação é a ausência de dados cadastrais dos clientes (ex: idade, renda, localização, profissão, score de crédito externo). Essas informações são, historicamente, as mais preditivas em modelos de risco.
-Definição de Inadimplência: A flag **inadimplente** é estática e aplicada a todas as transações de um cliente. Isso não captura a dinâmica temporal, ou seja, quando um cliente se tornou **inadimplente**. Um cliente pode ter um longo histórico de bom pagador antes de se tornar um risco.
Imputação de Nulos Simplificada: Assumir valor **0.0** para transações nulas é uma premissa que pode ser questionada. Uma análise mais detalhada da origem desses nulos poderia levar a uma estratégia de imputação mais sofisticada.

##3.2. Sugestões de Melhorias e Próximos Passos
-_Enriquecimento com Dados Externos_: A maior alavanca de melhoria seria enriquecer a tabela **default.clientes_features** com dados demográficos.
-Modelagem de Séries Temporais: Tratar o problema como uma análise de sobrevivência ou série temporal. Em vez de prever um status estático, o objetivo seria prever a probabilidade de inadimplência nos próximos X meses, com base no histórico de transações até o momento.
Features de Interação e Tendência: Criar features mais complexas, como a variação da média de gastos entre diferentes meses, ou a proporção do gasto em categorias de "risco" (identificadas na _EDA_) sobre o gasto total.
-_Construção do Modelo Preditivo_: Avançar para a etapa de modelagem, utilizando a tabela de features para treinar e validar múltiplos algoritmos (ex: Regressão Logística, Gradient Boosting, Redes Neurais) e implementar o campeão em um fluxo de produção.

####Observações Finais:
Durante o desenvolvimento no Databricks Free Edition, foi necessário adaptar as técnicas de manipulação de dados em função das limitações da plataforma. O acesso ao DBFS público é desabilitado, sendo assim, com suas limitações e forçando a usar Delta Tables, tendo também de usar a interface gráfica pra subir o .csv por meio do “Upload por UI”. Como os DataFrames utilizados são do Spark, que opera de forma distribuída, não foi possível usar diretamente bibliotecas tradicionais do Python, como seaborn ou scikit-learn. Para usá-las, seria necessário converter os dados para um DataFrame do Pandas, o que poderia comprometer o desempenho ou até inviabilizar a execução em casos de grandes volumes de dados.
Dessa forma, escolhi empregar funções nativas do Spark, como groupBy e pivot, que permitem realizar agregações, transformações e análises diretamente no motor de processamento distribuído. Essa abordagem garantiu a execução eficiente das etapas do projeto, além de manter a compatibilidade com o ambiente disponível no Databricks Free Edition. Além disso, o Databricks Free Edition, também não permite, em sua segurança/privacidade, o compartilhamento de Workspaces por link/convite na plataforma, essas são features da versão paga da plataforma.